# Create Dynamic Climate Indices

Notebook 1/X

This notebook was written by Logan Qualls. Data for this work is sourced from the National Center for Atmospheric Research's Catchment Attributes and Meterology for Large-Sample Studies (CAMELS) dataset, and this notebook is designed to work specifically with Frederik Kratzert's NeuralHydrology (NH; https://github.com/neuralhydrology/neuralhydrology) and Grey Nearing's SACSMA-SNOW17 (SAC-SMA; https://github.com/Upstream-Tech/SACSMA-SNOW17). NH provides a flexible framework with a variety of tools specifically designed for straightforward application of Long Short-Term Memory networks to hydrological modeling. The SACSMA-SNOW17 model provides a Python interface for the SAC-SMA model.

As climate change continues to impact our world, it becomes increasingly important to understand the robustness of our best (Long Short-Term Memory networks), and most commonly used (conceptual models like SAC-SMA), hydrological models. To begin characterizing model robustness we need to first create dynamic climate indices to serve as the independent variable of our experiments. This notebook will output a pickled dictionary containing dataframes for each basin with various dynamic climate indices.

### Import Libraries

In [1]:
import os
import supplemental
import pickle as pkl
from pathlib import Path
from supplemental import config
from supplemental.utils import load_basin_file
from supplemental.climateindices import calculate_camels_us_dyn_climate_indices

### Define Parameters

##### Most Important Experiment Parameters

First we need to define which forcing source we want to create dynamic climate indices from. Five forcing sources are avaliable through CAMELS, including daymet, maurer, maurer_extended, nldas, and nldas_extended. We also need to specify the length of the rolling window, as NeuralHydrology's calculate_camels_us_dyn_climate_indices uses a rolling mean window to calculate the dynamic climate indices.

In [2]:
#########################################################################################

#Specify which forcing source we want to create dynamic climate indices from
forcing = 'daymet'

#Specify mean rolling window length
window = 365

#########################################################################################

#If the dynamic climate indices file for the forcing, window, and basin list already exists, overwrite it?
overwrite = False

#########################################################################################

##### Paths

Next, we need to specify several paths. Example path endings are included above each requested path to help with this. Most should not have to be changed if this repositories native file structure is used.

In [3]:
#########################################################################################

#Working dir (current path; ../NeuralHydrology-Climate-Experiments)
working_dir = Path(os.getcwd())

#Path to config dir (../configs)
config_dir = working_dir / 'configs'

#Path to dynamic climate indices directory (../configs/dynamic_climate_indices)
dyn_clim_ind_dir = config_dir / 'dynamic_climate_indices'

#Path to dummy config file (../dummy_configs/climate_experiment_static_dummy.yml)
dummy_config_file = config_dir / 'dummy_configs' / f'climate_experiment_static_dummy.yml' 

#########################################################################################

**You should not have to edit anything below this cell.**

The cell below loads the dummy config file and retrieves the basin list file. Default is the 531_basin_list.txt file.

In [4]:
#########################################################################################

#Load config using NH's Config function
cfg = config.Config(dummy_config_file)

#Retrieve list of basins
basins = load_basin_file(cfg.train_basin_file)

#Get length of basins list
num_basins = len(basins)

#File path and name of output file; named according to forcing, number of basins, and window length variables
output_file = dyn_clim_ind_dir / f'dyn_clim_indices_{forcing}_{num_basins}basins_{window}.p'

#########################################################################################

### Calculate Dynamic Climate Indices

Finally, calculate dynamic climate indices for the specified forcing, basin list, and window length using NH's calculate_camels_us_dyn_climate_indices function.

In [5]:
#If the output file above does not already exist OR we want to overwrite the existing file...
if os.path.exists(output_file) == False or overwrite == True:

    #Calculate dynamic climate indices from CAMELS forcing data
    climate_indices = calculate_camels_us_dyn_climate_indices(data_dir=cfg.data_dir,
                                                              basins=basins,
                                                              window_length=window,
                                                              forcings=forcing,
                                                              output_file=output_file)

    #Save climate_indices to the specified file path and name
    with open(output_file,'wb') as f:
        pkl.dump(climate_indices, f)

100%|██████████| 531/531 [01:40<00:00,  5.29it/s]


In [6]:
#Take a peek at our new climate indices dataframe for an example basin
#.tail() is used because .head() often shows all NaNs, which is normal
climate_indices[basins[0]].tail()

,p_mean_dyn,pet_mean_dyn,aridity_dyn,t_mean_dyn,frac_snow_dyn,high_prec_freq_dyn,high_prec_dur_dyn,low_prec_freq_dyn,low_prec_dur_dyn
date,,,,,,,,,
2014-12-27,4.267370,1.250086,0.292941,6.250356,0.233348,0.049315,1.384615,0.627397,3.578125
2014-12-28,4.281918,1.250447,0.292030,6.277178,0.232555,0.046575,1.307692,0.624658,3.619048
2014-12-29,4.258164,1.250047,0.293565,6.284644,0.228274,0.049315,1.384615,0.627397,3.578125
2014-12-30,4.214740,1.249928,0.296561,6.283027,0.220323,0.052055,1.357143,0.630137,3.593750
2014-12-31,4.214740,1.249973,0.296572,6.291493,0.220323,0.052055,1.357143,0.630137,3.593750


Now that we have dynamic climate indices calculated, we can create configuration files.